## PyTorch Resnets

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from copy import deepcopy
import sys
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH
)
from experiments.utils.parser import Parser

In [2]:
series = 84
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = 'yolo'
series_path = os.path.join(
    NODE_PROFILING_RESULTS_PATH,
    'series',
    str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper,
    model_name=model_name)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

FileNotFoundError: [Errno 2] No such file or directory: '/home/cc/infernece-pipeline-joint-optimization/data/results/profiling/nodes/series/84/key_config_mapper.csv'

In [3]:
# results = loader.read_results(selected)
# final_dataframe = []
# for experiment_id, result in results.items():
#     processed_exp = {'experiment_id': int(experiment_id)}
#     flattened_results = self.flatten_results(
#         results[str(experiment_id)]['responses'])
#     if log is not None:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log[experiment_id])
#     else:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log)     

In [4]:
results.columns

NameError: name 'results' is not defined

In [12]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config) 

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1', '2', '4', '8'],
    'data_type': 'image',
    'max_batch_size': ['1'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x'],
    'node_name': 'yolo',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1],
    'series': 84,
    'series_meta': 'checking the effect of changing all variables under '
                   'arrival rate 1,\\n cpu type: Intel(R) Xeon(R) Gold 6126 '
                   'CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 60, 'loads_to_test': [1, 10, 20, 30]},
    'workload_type': 'static'}


In [13]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,yolo,yolov5n,1,10Gi,1,1,1,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
1,2,video,yolo,yolov5n,1,10Gi,1,1,10,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
2,3,video,yolo,yolov5n,1,10Gi,1,1,20,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
3,4,video,yolo,yolov5n,1,10Gi,1,1,30,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
4,5,video,yolo,yolov5n,2,10Gi,1,1,1,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,video,yolo,yolov5x,4,10Gi,1,1,30,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
76,77,video,yolo,yolov5x,8,10Gi,1,1,1,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
77,78,video,yolo,yolov5x,8,10Gi,1,1,10,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1
78,79,video,yolo,yolov5x,8,10Gi,1,1,20,60,84,checking the effect of changing all variables ...,1,False,exponential,image,1


In [17]:
experiment_ids = key_config_df['experiment_id'].tolist()
metadata_columns = ['model_variant', 'max_batch_size', 'cpu_request', 'load', 'load_duration', 'replicas']
results_columns = ['timeout_count', 'model_latencies_min', 'model_latencies_p99', 'cpu_usage_count_avg', 'model_latencies_avg', 'throughput_max']
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns)
output['total_requests'] = output['load'] * output['load_duration']
# output.sort_values(by='throughput')
# display(output[['model_variant', 'max_batch_size', 'cpu_request', 'load', 'load_duration', 'replicas', 'total_requests', 'timeout_count', 'model_latencies_p99', 'model_latencies_avg', 'throughput_max']][output['load'] == 80])
selected = output[['model_variant', 'max_batch_size', 'cpu_request', 'load', 'load_duration', 'replicas', 'total_requests', 'timeout_count', 'model_latencies_p99', 'model_latencies_avg', 'throughput_max']]

In [27]:
selected[selected['model_variant'] == 'yolov5x']

,model_variant,max_batch_size,cpu_request,load,load_duration,replicas,total_requests,timeout_count,model_latencies_p99,model_latencies_avg,throughput_max
64,yolov5x,1,1,1,60,1,60,0,1.293244,1.162854,0.227374
65,yolov5x,1,1,10,60,1,600,0,1.310166,1.164507,0.856140
66,yolov5x,1,1,20,60,1,1200,0,1.295726,1.154423,0.864345
67,yolov5x,1,1,30,60,1,1800,0,1.308016,1.152240,0.865999
68,yolov5x,1,2,1,60,1,60,0,0.959778,0.715598,0.983607
69,yolov5x,1,2,10,60,1,600,0,0.792083,0.694440,1.435367
70,yolov5x,1,2,20,60,1,1200,0,0.784726,0.694410,1.434845
71,yolov5x,1,2,30,60,1,1800,0,0.781309,0.695147,1.432722
72,yolov5x,1,4,1,60,1,60,0,0.483948,0.400021,1.000000
73,yolov5x,1,4,10,60,1,600,0,0.442808,0.385239,2.575071


In [23]:
selected['model_variant'].unique()

array(['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x'],
      dtype=object)